[View in Colaboratory](https://colab.research.google.com/github/tcglarry/2018-pixnet-hackathon/blob/master/start.ipynb)

In [1]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [3]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [28]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!ls drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option
aoi_aoi_test1.ipynb  Colab Notebooks	TAHU PowerPoint Template.pdf
aoi_test	     PiXNet		TAHU PowerPoint Template - 柱狀圖 1.ods
Capstone_PixNet.zip  Sample upload.txt	test for excel


In [0]:
# Start

In [32]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
import subprocess
import os
import pickle
from keras.layers import Input, Conv2D, MaxPooling2D, merge, Lambda,UpSampling2D, concatenate
from keras.models import Model
import pandas as pd
import sklearn 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [33]:
train_img = os.listdir('train_images')
len(train_img)

2528

In [0]:
img_list=[]
for i,img in enumerate(train_img):
    x = cv2.imread('drive/aoi_test/train_images/'+img, cv2.IMREAD_GRAYSCALE)
    img_list.append(x)

In [35]:
train_data = np.array(img_list)
train_data =  train_data[:,:,:,np.newaxis]
print (train_data.shape)

(2528, 512, 512, 1)


In [36]:
df_y = pd.read_csv('drive/aoi_test/train.csv')
df_y['Label'].value_counts()

0    674
5    644
1    492
3    378
4    240
2    100
Name: Label, dtype: int64

In [0]:
train_y =  df_y['Label'].values

In [38]:
onehot_encoder = OneHotEncoder(sparse=False)
train_y = train_y.reshape(len(train_y), 1)
print (train_y.shape)
train_label = onehot_encoder.fit_transform(train_y)


print ('beofre', train_label.shape)
print (df_y['Label'][:10])
print (train_label[:10,:])

train_label = train_label[:,np.newaxis,np.newaxis,:]

print ('after', train_label.shape)

(2528, 1)
('beofre', (2528, 6))
0    0
1    1
2    1
3    5
4    5
5    5
6    3
7    0
8    3
9    5
Name: Label, dtype: int64
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1.]]
('after', (2528, 1, 1, 6))


(2022, 512, 512, 1)
(2022, 1, 1, 6)
(506, 512, 512, 1)
(506, 1, 1, 6)


In [0]:
def conv_block(ch, activation= 'relu', padding='same'):       
    return Conv2D(ch,(3,3),activation= activation, padding =padding )


In [0]:
def build_model_1(ch=4, category=6):
    #inputs = Input((IMAGE_HEIGHT,IMAGE_WIDTH,ch))
    inputs = Input((512,512,1))
    inputs_norms = Lambda(lambda x : x/127.5 -1.0)(inputs)
    conv1 =  conv_block(ch)(inputs_norms)
    conv1 = conv_block(ch)(conv1)
    pool1 = MaxPooling2D(pool_size=(2,2))(conv1)
    
    conv2 =  conv_block(ch*2)(pool1)
    conv2 = conv_block(ch*2)(conv2)
    pool2 = MaxPooling2D(pool_size=(2,2))(conv2)
    
    conv3 =  conv_block(ch*4)(pool2)
    conv3 = conv_block(ch*4)(conv3)
    pool3 = MaxPooling2D(pool_size=(2,2))(conv3)
    
    
    conv4 =  conv_block(ch*8)(pool3)
    conv4 = conv_block(ch*8)(conv4)
    pool4 = MaxPooling2D(pool_size=(2,2))(conv4)
    
    conv5 =  conv_block(ch*16)(pool4)
    conv5 = conv_block(ch*16)(conv5)

    
    up6 = concatenate([UpSampling2D(size=(2,2))(conv5), conv4], axis=3)
    conv6 = conv_block(ch*8)(up6)
    conv6 = conv_block(ch*8)(conv6)
    
    up7 = concatenate ([UpSampling2D(size=(2,2))(conv6), conv3],  axis=3)
    conv7 = conv_block(ch*4)(up7)
    conv7 = conv_block(ch*4)(conv7)
    
    up8 = concatenate([UpSampling2D(size=(2,2))(conv7), conv2], axis=3)
    conv8 = conv_block(ch*2)(up8)
    conv8 = conv_block(ch*2)(conv8)
    
    up9 = concatenate([UpSampling2D(size=(2,2))(conv8), conv1],  axis=3)
    conv9 = conv_block(ch)(up9)
    conv9 = conv_block(ch)(conv9)
    
    
    
    conv10 = Conv2D(1,(1,1))(conv9)
    
    #model = Model(inputs= inputs, outputs = pool4)

    conv_11 =  conv_block(ch)(conv10)
    conv_11 = conv_block(ch)(conv_11)
    pool_11 = MaxPooling2D(pool_size=(2,2))(conv_11)

    conv_12 =  conv_block(ch*2)(pool_11)
    conv_12 = conv_block(ch*2)(conv_12)
    pool_12 = MaxPooling2D(pool_size=(2,2))(conv_12)

    conv_13 =  conv_block(ch*4)(pool_12)
    conv_13 = conv_block(ch*4)(conv_13)
    pool_13 = MaxPooling2D(pool_size=(2,2))(conv_13)


    conv_14 =  conv_block(ch*8)(pool_13)
    conv_14 = conv_block(ch*8)(conv_14)
    pool_14 = MaxPooling2D(pool_size=(2,2))(conv_14)

    conv_15 =  conv_block(ch*16)(pool_14)
    conv_15 = conv_block(ch*16)(conv_15)
    pool_15 = MaxPooling2D(pool_size=(2,2))(conv_15)

    conv_16 =  conv_block(ch*16)(pool_15)
    conv_16 = conv_block(ch*16)(conv_16)
    pool_16 = MaxPooling2D(pool_size=(2,2))(conv_16)

    conv_17 =  conv_block(ch*16)(pool_16)
    conv_17 = conv_block(ch*16)(conv_17)
    pool_17 = MaxPooling2D(pool_size=(2,2))(conv_17)

    conv_18 =  conv_block(ch*16)(pool_17)
    conv_18 = conv_block(ch*16)(conv_18)
    pool_18 = MaxPooling2D(pool_size=(2,2))(conv_18)

    conv_19 =  conv_block(ch*8)(pool_18)
    conv_19 = conv_block(ch*8)(conv_19)
    pool_19 = MaxPooling2D(pool_size=(2,2), padding='valid')(conv_19)


    conv_20 = Conv2D(category,(1,1),activation='softmax')(pool_19)

  
    
  
    model = Model(inputs= inputs, outputs=conv_20)
    
    return model

In [42]:
model = build_model_1()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 1)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 512, 512, 1)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 512, 512, 4)  40          lambda_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 512, 512, 4)  148         conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [47]:
import os
if os.path.isfile('drive/aoi_test/aoi.h5'):
  model = load_model('my_model.h5')
else:
  print ('not exist')
  


not exist


In [0]:
model.compile (optimizer='Adam', loss = 'categorical_crossentropy')

In [0]:
for i in range (100):
  if os.path.isfile('drive/aoi_test/aoi.h5'):
    model = load_model('drive/aoi_test/aoi.h5')
    print ('model laoded')
  else:
    print ('model not exist')
  train_x,val_x,train_y,val_y = train_test_split(train_data,train_label,test_size=0.2, shuffle=True)
  
  model.fit(x=train_x, y=train_y, batch_size=32, epochs=5, verbose=1, validation_split=0.2,  shuffle=True)
  model.save('drive/aoi_test/aoi.h5')

model not exist
Train on 1617 samples, validate on 405 samples
Epoch 1/5
